# Import library

In [1]:
!pip install google_play_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.2 MB/s eta 0:00:00


In [2]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.1 MB/s eta 0:00:00


In [1]:
!pip install gensim

In [2]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
seed = 0
np.random.seed(seed)
from sklearn.metrics import accuracy_score
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud
import nltk

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Load Dataset

In [4]:
# Membuat dataframe hasil scraping
from operator import index
review_mlbb =pd.read_csv('ulasan.csv')

# hitung jumlah baris dan kolom
jumlah_ulasan, jumlah_kolom = review_mlbb.shape

print(f"Jumlah ulasan: {jumlah_ulasan}")
print(f"Jumlah kolom: {jumlah_kolom}")

Jumlah ulasan: 81000
Jumlah kolom: 6


In [7]:
# Menampilkan 5 baris pertama dari review_mlbb
review_mlbb.head()

,userName,score,content,thumbsUpCount,reviewCreatedVersion,at
0,Pengguna Google,1,"Game buruk, system matchmaking game paling bur...",1368,2.1.18.11201,2025-10-04 00:03:18
1,Pengguna Google,1,"game ini semakin kesini makin gaje, setiap mai...",336,2.1.18.11201,2025-10-03 19:07:23
2,Pengguna Google,3,"overall udah bagus, tapi kadang lawan ga seimb...",144,2.1.18.11201,2025-10-04 11:04:50
3,Pengguna Google,5,Mobile Legends itu bagus banget dari segi perm...,544,2.1.18.11201,2025-10-05 20:46:22
4,Pengguna Google,1,"hallo admin mobile legend, diharapkan bisa mem...",1864,2.1.18.11201,2025-09-29 20:05:52


In [8]:
review_mlbb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81000 entries, 0 to 80999
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   userName              81000 non-null  object
 1   score                 81000 non-null  int64 
 2   content               81000 non-null  object
 3   thumbsUpCount         81000 non-null  int64 
 4   reviewCreatedVersion  69563 non-null  object
 5   at                    81000 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.7+ MB


In [9]:
# Cek data null
review_mlbb.isnull().sum()

,0
userName,0
score,0
content,0
thumbsUpCount,0
reviewCreatedVersion,11437
at,0


In [11]:
# Cek data duplikat
review_mlbb.duplicated().sum()

0

In [10]:
# Membuat dataframe baru dengan menghapus baris null di review_mlbb
mlbb = review_mlbb.dropna()

In [12]:
mlbb.isnull().sum()

,0
userName,0
score,0
content,0
thumbsUpCount,0
reviewCreatedVersion,0
at,0


In [13]:
jumlah_ulasan_update, jumlah_kolom_update = mlbb.shape

print(f"Jumlah ulasan: {jumlah_ulasan_update}")
print(f"Jumlah kolom: {jumlah_kolom_update}")

Jumlah ulasan: 69563
Jumlah kolom: 6


In [14]:
mlbb.head()

,userName,score,content,thumbsUpCount,reviewCreatedVersion,at
0,Pengguna Google,1,"Game buruk, system matchmaking game paling bur...",1368,2.1.18.11201,2025-10-04 00:03:18
1,Pengguna Google,1,"game ini semakin kesini makin gaje, setiap mai...",336,2.1.18.11201,2025-10-03 19:07:23
2,Pengguna Google,3,"overall udah bagus, tapi kadang lawan ga seimb...",144,2.1.18.11201,2025-10-04 11:04:50
3,Pengguna Google,5,Mobile Legends itu bagus banget dari segi perm...,544,2.1.18.11201,2025-10-05 20:46:22
4,Pengguna Google,1,"hallo admin mobile legend, diharapkan bisa mem...",1864,2.1.18.11201,2025-09-29 20:05:52


# Preprocessing Teks

Cleaning

In [15]:
def cleaningTeks(teks):
    teks = re.sub(r'@[A-Za-z0-9]+', '', teks)
    teks = re.sub(r'#[A-Za-z0-9]+', '', teks)
    teks = re.sub(r'RT[\s]', '', teks)
    teks = re.sub(r'http\S+', '', teks)
    teks = re.sub(r'[0-9]+', '', teks)
    teks = re.sub(r'[^\w\s]', '', teks)
    teks = teks.replace('\n', ' ')
    teks = teks.translate(str.maketrans('', '', string.punctuation))
    teks = teks.strip(' ')
    return teks

Mengubah semua jadi huruf kecil

In [16]:
def caseFoldingTeks(teks):
    teks = teks.lower()
    return teks

Tokenization

In [17]:
def tokenizingTeks(teks):
    teks = word_tokenize(teks)
    return teks

Menghapus Stopwords

In [18]:
def filteringTeks(teks):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku', 'yang', 'ke',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for tks in teks:
        if tks not in listStopwords:
            filtered.append(tks)
    tks =  filtered
    return teks

Stemming

In [19]:
def stemmingTeks(teks):
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = teks.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_teks = ' '.join(stemmed_words)

    return stemmed_teks

Sentence

In [20]:
# Mengubah daftar kata menjadi kalimat
def toSentence(list_words):
    if not list_words:
        return ''
    return ' '.join(word for word in list_words)

In [21]:
slangwords = {'@' :'di', "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", 'kalo' : 'kalau', 'yg' : 'yang'}
def fix_slangwords(teks):
    words = teks.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_teks = ' '.join(fixed_words)
    return fixed_teks

In [22]:
# Membersihkan teks dan menyimpannya di 'teks_bersih'
mlbb['teks_bersih'] = mlbb['content'].apply(cleaningTeks)

# Mengubah semua huruf dalam teks menjadi huruf kecil dan menyimpannya di 'teks_casefolding'
mlbb['teks_casefolding'] = mlbb['teks_bersih'].apply(caseFoldingTeks)

# Mengganti kata kata slang dengan kata kata standar dan menyimpannya di 'teks_slangwords'
mlbb['teks_slangwords'] = mlbb['teks_casefolding'].apply(fix_slangwords)

# Membagi kata kata menjadi token dan menyimpannya di 'teks_tokenizing'
mlbb['teks_tokenizing'] = mlbb['teks_slangwords'].apply(tokenizingTeks)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
mlbb['teks_stopword'] = mlbb['teks_tokenizing'].apply(filteringTeks)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
mlbb['teks_akhir'] = mlbb['teks_stopword'].apply(toSentence)

# Pelabelan

In [23]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_positive[row[0]] = int(row[1])

else:
        print('Gagal mengambil lexicon positive')

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_negative[row[0]] = int(row[1])

else:
        print('Gagal mengambil lexicon negative')

In [24]:
# Fungsi untuk menentukan polaritas sentimen dari tweet
def analisis_sentimen(teks):
    score = 0

    for word in teks:
        if word in(lexicon_positive):
            score = score + lexicon_positive[word]

    for word in teks:
        if word in(lexicon_negative):
            score = score + lexicon_negative[word]

    polarity =''

    if(score > 0):
        polarity = 'positive'
    elif(score < 0):
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity

In [25]:
results = mlbb['teks_stopword'].apply(analisis_sentimen)
results = list(zip(*results))
mlbb['polarity_score'] = results[0]
mlbb['polarity'] = results[1]
print(mlbb['polarity'].value_counts())

polarity
negative    56584
positive    10749
neutral      2230
Name: count, dtype: int64


# Ekstraksi Fitur dan Penerapan Model Machine Learning

TF-IDF

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = mlbb['teks_akhir']
y = mlbb['polarity']

tfidf = TfidfVectorizer(max_features=200, min_df=7, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

In [27]:
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [28]:
from sklearn.linear_model import LogisticRegression

# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression()

# Melatih model Logistic Regression pada data pelatihan
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic Regression pada data pelatihan
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)

# Evaluasi akurasi model Logistic Regression pada data uji
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# Menampilkan akurasi
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.8826954177897575
Logistic Regression - accuracy_test: 0.8832027600086251


In [6]:
!jupyter nbconvert --to python /content/AnalisisSentimen.ipynb

[NbConvertApp] Converting notebook /content/AnalisisSentimen.ipynb to python
[NbConvertApp] Writing 7938 bytes to /content/AnalisisSentimen.py


In [7]:
!pip install pipreqs

In [8]:
!pipreqs /content

INFO: Not scanning for jupyter notebooks.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
Please, verify manually the final list of requirements.txt to avoid possible dependency confusions.
INFO: Successfully saved requirements file in /content/requirements.txt
